In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# 回帰

オレンジジュースのデータを用いて、線形モデルの取り扱いについて学ぶ。

In [ ]:
oj = pd.read_csv("https://raw.githubusercontent.com/TaddyLab/BDS/cea195208c1a3d06405b6b404277e1b63eb38e6c/examples/oj.csv")

In [ ]:
# oj.to_csv("oj.csv", index=False)
# oj = pd.read_csv("oj.csv")

データの可視化

In [ ]:
oj["log_price"] = np.log(oj.price)

In [ ]:
# 描画の都合上、データ数を1000にサンプリングしている
px.strip(oj.sample(n=1000), x="brand", y="log_price", color="brand", height=400, width=500)

In [ ]:
px.violin(oj, x="brand", y="log_price", color="brand", height=400, width=500)

In [ ]:
px.box(oj, x="brand", y="log_price", color="brand", height=400, width=500)

分布の形状を掴むのにはviolin plotが一番見やすいかな。  
ホバーすればちゃんとmedianを教えてくれる。

In [ ]:
oj["log_move"] = np.log(oj.sales)

In [ ]:
# # 参考、plotlyでサクッと回帰する場合
# px.scatter(
#     oj, x="log_price", y="log_move", color="brand",
#     symbol_sequence=["circle-open"], opacity=0.1, range_x=[-0.5, 1.5],
#     trendline="ols", render_mode="webgl", # データ数が1000を超えるときはwebglの方がいいらしい。
#     height=400, width=500)

対数売上数と対数価格は大まかな線形関係にある。  

目的変数に対数を取って整理すると以下のようになる。

$$
log(y) = \alpha + \beta x \\
y = \exp^{\alpha} \exp^{\beta x} \\
$$

$x^{*} = x + 1$とすると、上記式は以下のようになる

$$
y^{*} = \exp^{\alpha} \exp^{\beta x^{*}} \\
y^{*} = \exp^{\alpha} \exp^{\beta x + \beta} \\
y^{*} = \exp^{\beta}\exp^{\alpha} \exp^{\beta x} \\
y^{*} = \exp^{\beta}y \\
$$

従って、$x$が1単位増えると$y$は$\exp^{\beta}$倍される。

目的変数、説明変数両方に対数を取ると、$\beta$は弾力性となる。  
$x$が$n$%増えると$y$は$\beta n$%変化する。

In [ ]:
# statsmodelsを用いた回帰
from statsmodels.formula.api import ols

In [ ]:
formula = "np.log(sales) ~ np.log(price) + brand"
reg = ols(formula=formula, data=oj).fit()
print(reg.summary())

上の推定結果では、消費者の価格感度が等しいと仮定している。  
ブランドによって消費者の価格感度が異なるとしたければ、交互作用項をいれる。

In [ ]:
formula = "np.log(sales) ~ np.log(price)*brand"
reg = ols(formula=formula, data=oj).fit()
print(reg.summary())

価格弾力性の比較は、  
ドミニクス: -3.38  
ミニッツメイド: -3.32  
トロピカーナ: -2.71  
となる。

最後に、広告の効果を見る。  
ブランド×広告有無で全ての交互作用行を加える。  
係数は、2(定数項+係数)×3(ブランド数)×2(広告有無)=12個になる。

In [ ]:
formula = "np.log(sales) ~ np.log(price)*brand*feat"
reg = ols(formula=formula, data=oj).fit()
print(reg.summary())

広告を打つと、価格感度がより敏感になる。  
元々価格感度が鈍かったトロピカーナやミニッツメイドの方が、広告を打った際の価格感度が高まっている。  
広告を打つことで、元々のブランド支持層以外が購買するため、価格感度が高まった。  
マーケティング概論の授業では、広告は値引とセットで行うべきというのはこれが理由。  

また、宣伝される品物は値引きを伴うため、需要曲線が線形でない(低価格だと価格感度が高い)という事もありうる。  

ミニッツメイドは広告を考慮しない場合価格感度がドミニクスに近かったが、広告を考慮するとトロピカーナに近い。  
ミニッツメイドは頻繁に広告されるため、広告を考慮しないと価格感度が不自然に高く見えてしまっていた(交絡)。  

変数の影響は正しく統制しなければならない。

In [ ]:
from statsmodels.graphics.mosaicplot import mosaic

In [ ]:
mosaic(oj, ["feat", "brand"]);

トロピカーナと比べ、ミニッツメイドは広告をたくさん打っていることがわかる。

# ロジスティック回帰

対数オッズの線形モデル

In [ ]:
from statsmodels.formula.api import logit

In [ ]:
email = pd.read_csv("https://raw.githubusercontent.com/TaddyLab/BDS/cea195208c1a3d06405b6b404277e1b63eb38e6c/examples/spam.csv")

In [ ]:
# email.to_csv("spam.csv", index=False)
# email = pd.read_csv("spam.csv")

In [ ]:
email.head()

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [ ]:
family = sm.families.Binomial()

In [ ]:
formula = "spam ~ " + "+".join([col for col in email.columns if col != "spam"])
res = smf.glm(formula=formula, data=email, family=family).fit()
print(res.summary())

In [ ]:
res.params.word_free, np.exp(res.params.word_free)

word_freeになるとスパムになるオッズが4.68倍になる。  
例えば、元々確率が10%だったのであればオッズは1/9なので、それが4.68/9=50%まで上昇する。

In [ ]:
res.params.word_george, np.exp(res.params.word_george)

word_georgeが含まれていると、オッズは300分の1未満になる。

In [ ]:
email.loc[[0,3999]]

In [ ]:
# 予測
res.predict(email.drop(["spam"], axis=1).loc[[0,3999]])

In [ ]:
formula = "spam ~ " + "+".join([col for col in email.columns if col != "spam"])
res = smf.logit(formula=formula, data=email).fit()

In [ ]:
print(res.pred_table(threshold=0.5))

行がTrue/False、列がPositive/Negative

In [ ]:
pred_prob = res.predict(email.drop(["spam"], axis=1))

In [ ]:
px.violin(pd.DataFrame({"type":email.spam, "pred_prob":pred_prob}), x="type", y="pred_prob", height=400, width=500)

In [ ]:
print(res.summary())

In [ ]:
res.aic, res.bic

回帰における不確実性  

係数の不確実性を捉える上で、分散不均一性がある場合、通常の標準誤差は誤った値となる。  
その場合は、ブートストラップ標本を用いて推定した係数の分布を実際に標本分散の推定値として使用できる。  
OLS回帰の特別な場合、頑健な標準誤差を取得するために、サンドウィッチ分散推定量を使える。

In [ ]:
import pyper

In [ ]:
r = pyper.R(use_pandas=True, use_numpy=True)

In [ ]:
print(r("data(airquality)"))

In [ ]:
r("fit <- glm(Ozone ~ ., data=airquality)")

In [ ]:
print(r('summary(fit)$coef["Wind", ]'))

Windの標準誤差は0.64

In [ ]:
r('library(AER)');

In [ ]:
r("bvar <- vcovHC(fit)");

In [ ]:
print(r("round(bvar, 1)"))

In [ ]:
print(r('sqrt(bvar["Wind", "Wind"])'))

WindのHC標準誤差は0.91, 大幅に大きくなった。

ブートストラップを用いて標準誤差を求める

In [275]:
# NAがあるとdata.frameをうまく引っ張ってこれない
# 先に置換して後から戻す
r("airquality[is.na(airquality)] <- -100")
airquality = pd.DataFrame(r.get('airquality'))

In [276]:
airquality.columns = [col.strip(" ").replace(".", "_") for col in airquality.columns]
# floatがfloat128になってしまっているので、適切な型に直す
# 直さないとpx.scatterで描画が出来ない
airquality = airquality.astype(np.float64).replace(-100, np.nan)
airquality["Temp"] = airquality["Temp"].astype(np.int64)
airquality["Month"] = airquality["Month"].astype(np.int64)
airquality["Day"] = airquality["Day"].astype(np.int64)

In [281]:
# airquality.to_csv("airquality.csv", index=False)

In [282]:
airquality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Ozone    116 non-null    float64
 1   Solar_R  146 non-null    float64
 2   Wind     153 non-null    float64
 3   Temp     153 non-null    int64  
 4   Month    153 non-null    int64  
 5   Day      153 non-null    int64  
dtypes: float64(3), int64(3)
memory usage: 7.3 KB


In [270]:
airquality.head()

,Ozone,Solar_R,Wind,Temp,Month,Day
0,41.0,190.0,7.4,67,5,1
1,36.0,118.0,8.0,72,5,2
2,12.0,149.0,12.6,74,5,3
3,18.0,313.0,11.5,62,5,4
4,-100.0,-100.0,14.3,56,5,5


In [285]:
B = 1000
betas = []
for b in range(B):
    samp_b = airquality.sample(frac=1, replace=True)
    res = ols(formula="Ozone ~ Solar_R + Wind + Temp + Month + Day", data=samp_b).fit()
    betas.append(res.params.Wind)

In [286]:
np.mean(betas), np.std(betas)

(-3.3579753773972354, 0.884377533188488)

不均一分散を考慮したHC標準誤差に近い結果を得た。

実際に散布図を確認してみると、風速と残差のばらつきが不均一になっていることがわかる。

In [287]:
res = ols(formula="Ozone ~ Solar_R + Wind + Temp + Month + Day", data=airquality).fit()

wind_res = pd.DataFrame({
    "wind": airquality.Wind,
    "OLS_res": res.resid
})

In [288]:
px.scatter(wind_res, x="wind", y="OLS_res", height=400, width=500)

風が弱いと、残差のばらつきが大きくなっていることがわかる。

これまでに説明した不確実性推定量は、観測値間の独立性を前提としている。  
そのため、HCの手順においても、$\hat{\Sigma}$の非対角線上は0になる。  

観測値間に従属関係がある場合に使用できるツールが他にある。  

従属関係を考慮したサンドウィッチ推定量(クラスタ化標準誤差)については5章で説明する。  
このような手段はランダムか比較試験で処置効果を推定するときに一般的に使用される。  

また、従属関係は回帰においてモデル化しても扱える。

時系列は、、まあ大体わかってるからパス  
先に進む